<a href="https://colab.research.google.com/github/nicolaiberk/bild/blob/main/code/content_analysis/bert/crime/bert/crime_annotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 34.6 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
     |████████████████████████████████| 6.6 MB 66.3 MB/s 
     |████████████████████████████████| 880 kB 68.7 MB/s 
     |████████████████████████████████| 77 kB 8.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d65a467d14e12838fb1a16bd94ba094dda253e680e2188b10dd99d7973e90ff6
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
import pandas as pd
import csv
import sys
import numpy as np


# parameters
clsfr_path = "drive/MyDrive/Bild/crime_clsfr_BERT_torch"
input_path = "https://www.dropbox.com/s/ousyjlju18fopny/_migration_articles_BERT.csv?dl=1"
output_path = "drive/MyDrive/Bild/crime_annotations.csv"
batch_size = 16

# load classifier and tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
model = AutoModelForSequenceClassification.from_pretrained(clsfr_path)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-german-cased', model_max_length = 512)
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, device = 0, padding='max_length', truncation=True)

Sanity Check:

In [ ]:
# Test model
text = "Die Polizei hat gestern zwei Flüchtige nach einem Bankraub festgenommen. Nach einem kurzen Schusswechsel konnten die mutmaßlichen Täter überwältigt und festegnommen werden. Die Staatsanwaltschaft ermittelt."
classifier(text)

[{'label': 'LABEL_1', 'score': 0.9991445541381836}]

In [ ]:
text = "Schleswig-Holstein hat einen neuen Landtag gewählt. Mit dem vorläufigen Endergebnis kommt die CDU als stärkste Kraft auf 43,4 Prozent der Stimmen, die SPD erreicht 16 Prozent, die Grünen sind zweitstärkste Kraft mit 18,3 Prozent."
classifier(text)

[{'label': 'LABEL_0', 'score': 0.9996545314788818}]

Looking good.

In [ ]:
# check if output file exists and count lines
try:
  temp = pd.read_csv(output_path)
  skip = pd.shape[0]
  del temp
except:
  skip = 0

In [ ]:
mig_articles = pd.read_csv(input_path)

In [ ]:
mig_articles.head(1)

,date,paper,title,url,text
0,15707,taz,Ländle ärgert sich über Lästerei,https://taz.de/Archiv-Suche/!527952&s=&SuchRah...,BERLIN | Nach den Lästereien von Bundestagsviz...


In [ ]:
mig_articles["crime"] = classifier([article for article in mig_articles.text])

In [ ]:
import re
int(re.match('LABEL_(\d)', 'LABEL_0').group(1))


0

In [ ]:
# clean
labels = []
prob = []
for dt in mig_articles.crime:
  labels.append(int(re.match('LABEL_(\d)', dt['label']).group(1)))
  prob.append(float(dt['score']))

mig_articles['crime_label'] = labels
mig_articles['crime_prob'] = prob

In [ ]:
mig_articles.to_csv(output_path)

In [ ]:
mig_articles.head()

,date,paper,title,url,text,crime,crime_label,crime_prob
0,15707,taz,Ländle ärgert sich über Lästerei,https://taz.de/Archiv-Suche/!527952&s=&SuchRah...,BERLIN | Nach den Lästereien von Bundestagsviz...,"{'label': 'LABEL_0', 'score': 0.9999115467071533}",0,0.999912
1,15707,taz,Die Kinder bleiben unter sich,https://taz.de/Archiv-Suche/!5076321&s=/,QABB ELIAS taz | Der elfjährige Firas wird vie...,"{'label': 'LABEL_0', 'score': 0.9999068975448608}",0,0.999907
2,15707,taz,Auf der Suche nach Vorbildern,https://taz.de/Archiv-Suche/!528028&s=&SuchRah...,Als „sehr positiv“ bewertet der Senat die Erfa...,"{'label': 'LABEL_0', 'score': 0.9999082088470459}",0,0.999908
3,15707,taz,Wenig arbeitslose Jugendliche,https://taz.de/Archiv-Suche/!528056&s=&SuchRah...,Die Jugendarbeitslosigkeit scheint in Niedersa...,"{'label': 'LABEL_0', 'score': 0.9998371601104736}",0,0.999837
4,15706,taz,„Jesus setzte sich für Fremde ein“,https://taz.de/Archiv-Suche/!5076305&s=/,Mehrere Dutzend Personen eines Kollektivs zur ...,"{'label': 'LABEL_0', 'score': 0.9998641014099121}",0,0.999864
